<a href="https://colab.research.google.com/github/QidiLiu/DL_by_Hand/blob/main/01_DL_Basics/01_04_%E6%95%B8%E6%93%9A%E6%93%8D%E4%BD%9C%E8%88%87%E9%A0%90%E8%99%95%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 數據操作

機器學習和神經網絡的基本數據結構是**N維數組**

創建數組需要：

- 尺寸
- 數據類型
- 初始化各個元素的值（全設爲0，或設爲隨機數）

訪問元素可以以坐標或坐標切片形式進行，如：

- a[1, 2]
- a[1, :]
- a[:, 1]
- a[1:3, 1:]
- a[::3, ::2] （步長大於0）

In [1]:
import torch

張量（tensor）表示一個多維數組

In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

可用shape訪問其尺寸，可用numel()訪問元素的總數

In [3]:
print(x.shape)
print(x.numel())

torch.Size([12])
12


用改變尺寸可用reshape()

In [4]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

可創建全0或全1的指定尺寸張量

In [5]:
print(torch.zeros((2, 3)))
print(torch.ones((2, 3, 2)))

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]])


可用列表直接創建張量（不加小數點會自動設爲整數型）

In [6]:
torch.tensor([[2, 1, 3], [4, 6, 8]])

tensor([[2, 1, 3],
        [4, 6, 8]])

直接對張量進行加減乘除可以對各個**對應**元素進行計算

In [7]:
x = torch.tensor([1.0, 2.0, 4.0, 8.0])
y = torch.tensor([2.0, 2.0, 2.0, 2.0])
x + y, x - y, x * y, x / y, x ** y

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

cat()用於堆叠張量，要注意堆叠的維度

In [8]:
X = torch.arange(12, dtype=torch.float32).reshape((3, 4))
Y = torch.tensor([[2., 1., 4., 3.], [1., 2., 3., 4], [4., 3., 2., 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

邏輯運算符會按元素求解

In [9]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

用sum()可求張量内元素的總和

In [10]:
X.sum()

tensor(66.)

廣播機制（粗淺解釋）：張量尺寸不同時，同維度的各尺寸會自動按最大的廣播后再做計算（雖然好用，但也容易出錯）

In [12]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]), tensor([[0, 1]]))

In [13]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

### 數據處理

創建一個數據集，並儲存在csv（逗號分隔值）文件

In [29]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file_path = os.path.join('..', 'data', 'house_tiny.csv')

with open(data_file_path, 'w') as file_writer:
    file_writer.write('NumRooms,Alley,Price\n')
    file_writer.write('nan,Pave,127500\n') # NA意爲未知量
    file_writer.write('2,nan,106000\n')
    file_writer.write('4,nan,178100\n')
    file_writer.write('nan,nan,140000\n')

用pandas可讀取剛剛創建的數據集

In [30]:
import pandas as pd

data = pd.read_csv(data_file_path)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


對於空缺數據，可做插值或刪除處理，此處演示插值

In [31]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


<ipython-input-31-7f3bcd4dc11f>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


將類別轉爲編碼（此處爲dummy編碼）

In [32]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


將數據轉爲張量

In [33]:
X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

### Q&A

- 不能用一個張量去初始化另一個張量，因爲并不是克隆，它們將指向同一片内存空間